# Python library for loading a log of the Consumer Physics SCIO


In [1]:
import json
import struct
import logging
import datetime
import os
import csv
import base64
import numpy as np
import re

# Logging/output format setup
log = logging.getLogger('root')
log.setLevel(logging.DEBUG)
logging.basicConfig(format='[%(asctime)s] %(levelname)8s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

In [62]:
class scio_log:
    def __init__(self):
        self.device_info = {}
        self.scan_rawdata = {}
        self.calibration_data = {}
        self.key = b''
        self.cmd_temp = r"\[command: (\w+), data: ([\w\s]+)\]"
        #self.cmd_info = r"DeviceInfo{(?P<info>.+?)}"
        self.cmd_info = r"DeviceInfo{(.*?)}"
    
    def load_log(self, log_fn, silent=True):
        next_task = 'dev_info'
        
        with open(log_fn, 'r') as file:
            for line in file:
                # Device information
                if(next_task == 'dev_info'):
                    match = re.search(self.cmd_info, line)
                    if(match):
                        self.parse_device_info(match)
                        next_task = 'temp_before'
                # Temperature before the scan
                if(next_task == 'temp_before'):
                    match = re.search(self.cmd_temp, line)
                    if(match):
                        temp = self.read_temperature(match)
                        temp_before_found = True
                        print(temp)
        
        print(self.device_info)
        return
    
    def parse_device_info(self, match):
        info = str(match).split(", ")
        print(info)
        for item in info:
            key, value = item.split("=")
            if value.isdigit():
                self.device_info[key] = int(value)
            elif value.lower() == "true":
                self.device_info[key] = True
            elif value.lower() == "false":
                self.device_info[key] = False
            else:
                self.device_info[key] = value
        return(self.device_info)
    
    def read_temperature(self, match):
        response_data_raw = bytearray.fromhex(match.group(2).replace(" ", ""))
        response_length = len(response_data_raw)
        # Parse response
        num_vars = response_length / 4 # divide by 4 because we are dealing with longs
        data_struct = '<' + str(int(num_vars)) + 'L' # This is usually '<3l' or '<lll'
        # Convert bytes
        response_data = struct.unpack(data_struct, response_data_raw)
        # Convert to actual temperatures
        cmosTemperature   = (response_data[0] - 375.22) / 1.4092 # From the disassembled Android app
        chipTemperature   = response_data[1] / 100
        objectTemperature = response_data[2] / 100
        temperatures = {'cmos_t': cmosTemperature, 'chip_t': chipTemperature, 'obj_t': objectTemperature}
        return(temperatures)
    
    # Saves a single scan as raw data to JSON
    def write_data_file(self, output_fn, scan_rawdata, temp_before, temp_after):
        timestamp = datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
        
        # Rename temperature reading dicts
        temp_before['cmos_t_before'] = temp_before.pop('cmos_t')
        temp_before['chip_t_before'] = temp_before.pop('chip_t')
        temp_before['obj_t_before']  = temp_before.pop('obj_t')
        temp_after['cmos_t_after']   = temp_after.pop('cmos_t')
        temp_after['chip_t_after']   = temp_after.pop('chip_t')
        temp_after['obj_t_after']    = temp_after.pop('obj_t')
        
        # Create the json string
        jsondata = {}
        jsondata['device'] = self.device_info
        jsondata['scan'] = {'timestamp': timestamp}
        jsondata['scan'].update(temp_before)
        jsondata['scan'].update(temp_after)
        jsondata['scan'].update(scan_rawdata)
        # Write data to JSON file
        with open(output_fn, 'w') as outfile:
            json.dump(jsondata, outfile, indent=4)
        return
    
    def encode_b64(self, bytestring):
        urlSafeEncodedBytes = base64.urlsafe_b64encode(bytestring)
        urlSafeEncodedStr = str(urlSafeEncodedBytes, 'utf-8')
        return(urlSafeEncodedStr)

In [63]:
log_fn  = './01_rawdata/log_files/log_20200604-bark.txt'
data_fn = './scan.json'

# Now set up device
scio = scio_log() # Create an instance for this device address
scio.load_log(log_fn, silent=False)
print('Done...')

['<re.Match object; span=(66', '180)', 'match="DeviceInfo{address=\'B4:99:4C:59:66:01\'', 'version=0>']


ValueError: not enough values to unpack (expected 2, got 1)

In [66]:
response_length = struct.unpack('>H',bytearray.fromhex('0290'))[0]
print(response_length)

656


In [68]:
656*2+8

1320

In [70]:
b64 = "AAAAAJkNjhaywcv3MpJ9AwH3AEL4QdS\/cA1EkZvkzf2eEUOZEoTFBPtNagKTfghSyZR470qmx7kB\nKh\/v4tR0aRDt6w3UOHrUsx65AGnVD5Du5DLsrPfemHBYBSkaC9ozPBT2Ss03rKG1cX9DLTYUHkcP\nISFsQe7FqS9iMEEJNfPR4LMe2hL5Qd0knSIcB8dCjup4nhwK2aNOy2PmE4gruIJOdRVHw1yf9eSw\nfQgQiqKSdDNtJEiiop2IGY21RoJP\/vyjZrgTOt7wxWnzSDj5IK8ohhB3EfYTUFNL0b8ROZ6aEv6P\nolRvI5iTv\/cARKgQ2VD7WZdckPUKVwF2esaTJ4n4kAbRmyqx1CyDN8Luesv486OI0AWcH5oz+iJG\nTN3QEofY8JRvUuS70GPndQPLTzEs6oNqNQuBX6im2kBrDszFQtzPqfrBlPnyw4+lIZybDlX9LZvF\nw\/fcy33uZIt3xjfb7\/wxP3tQH8PaIXm5LAzuYBCq45EpOSpMjQjpCOYsNGVeAMRzlH2oa3LeXXkD\nlfC4Eru0y56f3DN1ToDRclKJRLXnoSHwHN2ORyCAiw7P3+hKwTuyCzlQrg4TcY8ZWfHIJlydBbnQ\nDVysZbbS8qnBw7h4coPOU1J\/rjGViDZQ+37id8d+9xYxvoDJR6rJ\/4xtziT3p12cXiLzKKdsGDO8\ntFQ8o7WoOcwSXfZWjR2ktm0P3a3D1V11itPIwd235qSGi2lqZXfDVao0cI1gaAxajeyOUsxuUdqU\niczSw2XgfghShXsyUU6bR5dT04J8vnxU49Hnxx669663VN8FS2jy9eITWsIvfjrMrXAxm4y4EnXn\nrtUnjEDQ\/D2E7uOrjS2VL91Iy7IU5mATmlYbTVEneW8B9APbo1o9thFUTgUCKg1JjzZvYoURviiL\n\/zVxpFaHcG5kELE8UwEMCRs7YaMhqc+k33jKYEeeEnGKIYNLDNV+YmqMWvezUcwyhcDJueuAvGws\nuA9Awa17Yoa10jJsDdW\/IXrx\/UzGRuNPJIaM46Lajn5KR5GxllJK4tIT8eQuXTQZHptd48cpJFVa\nX7hx0wYlQRCHmG4Ua52S62SQSIxUo512Eu4F0zhe9rfa0rrAvbgd9VK7dXGcGYxPouYkPw+AfGho\nl0aNVHiahxTuD0\/2yMpiDpAtTvb+s08LUWLjhe5RYQ9Z1Fdagi3Id5XnX65R4MwQNMBCBsVxQsv4\nubripUt1AQqAAFmtrYlc+tepgfR0D+7t8E5+5KT\/YgWTwDivg5x8xjfFpeRyrS8qR1Wfvqd9zc6g\nnE0KfvbSwWPizMKeLFhOlOHjHveTS3Yr7D7U6Rrds6ltwRb9sr0RBtL1AaaYcCIg8bAYAocfvKDy\ndFx0XfOX1LYTelW2DyTAElO9MjUcQbh\/UNXxbvNU7QkXtPk9z9YhATRTZVnPTMXx7ScK6MpYmCG7\nfp0gKuPj7ylsxr5RgT2fmQPbpOBHhavi1MhIefqJqZxZSJTEYaKPZK2PO2N+8lSRUiz76EeKkL8g\n7VBGEDX7JwWy5fgrCNcBWRrFOO49OYAAJnd2Dme\/nMHVK3rgvITGhNNRAlyJ\/jVQHLlP2uYLwoHm\nSddcomXQryrC2O6HDni\/op\/uQsDt3XC21UWLHiUQV8ILVGumhdkf6\/x8rnIgc2NE35PQO2OyFEGd\nIOFf3d\/IU9NBqKlEf7j\/Oc3RMFq4vx1ZicheCOp2NNKwEUG8b02WmuhMOMTLqwW571aatNv\/fVci\n0KwbGXSAjRNgKMj2zmzf4OvPqfHuqo91DUh5bdK3QSinznec0QW39nSl7iiNqPcwqJGO8fcKFGTG\nwCQ1NwR57jYxf5ju4yCyscDIKrKdvGh7wf7hMv+ehFQDkfx5ixnoAiw0MzaIpCW25jT6B5s+Kucz\nnJ1kqMvSk4huOpkAk3k5ITHzW7CkIqnfxTrM+e6vW5HJ8ne7tkJox6gXG7iXRhakmYgyVa6nf1W5\nzAglWa2fkdw9aEAAHyyRtgHUL9l8OrpAqTioB1\/JiMjcxswOowgI8\/qlx2A0kdANuBR0NaeIndlQ\ns7MhDz1h\/tQJQ3ogpnuVW+4\/4QF6vIzl7cxC4O+8mVGO0uL0qP2WcrHI3cyKq0eJNAiCITeANiZT\nK49odoYbzbOfcqOKIFxF23MMwp+21mYBYc3mp\/6kbZJa85tkr++CPh38jeqTcg0JsE9lus48+eGW\n90Fe2X+0\/ewBtboniGPdtm5\/KTCMRXkhLVMWBAPR\/rbHRCGb75fbzyRUM+msW9b925LKryQWqas1\n+H2E0xGGfixqPPgm3zVH0ATLw3KuNmYZv\/7LibkY9fQsD4yimWdNwL181fONJ8D1AyKCk4cfeTW5\nmAg3mf6SBIxXgIktZ2HdOSvw0DJxBUFLdP69gzrb0f\/M\n"

byte_str = base64.b64decode(b64).hex()

print(byte_str)

00000000990d8e16b2c1cbf732927d0301f70042f841d4bf700d44919be4cdfd9e1143991284c504fb4d6a02937e0852c99478ef4aa6c7b9012a1fefe2d4746910edeb0dd4387ad4b31eb90069d50f90eee432ecacf7de98705805291a0bda333c14f64acd37aca1b5717f432d36141e470f21216c41eec5a92f6230410935f3d1e0b31eda12f941dd249d221c07c7428eea789e1c0ad9a34ecb63e613882bb8824e751547c35c9ff5e4b07d08108aa29274336d2448a2a29d88198db546824ffefca366b8133adef0c569f34838f920af2886107711f61350534bd1bf11399e9a12fe8fa2546f239893bff70044a810d950fb59975c90f50a5701767ac6932789f89006d19b2ab1d42c8337c2ee7acbf8f3a388d0059c1f9a33fa22464cddd01287d8f0946f52e4bbd063e77503cb4f312cea836a350b815fa8a6da406b0eccc542dccfa9fac194f9f2c38fa5219c9b0e55fd2d9bc5c3f7dccb7dee648b77c637dbeffc313f7b501fc3da2179b92c0cee6010aae39129392a4c8d08e908e62c34655e00c473947da86b72de5d790395f0b812bbb4cb9e9fdc33754e80d172528944b5e7a121f01cdd8e4720808b0ecfdfe84ac13bb20b3950ae0e13718f1959f1c8265c9d05b9d00d5cac65b6d2f2a9c1c3b8787283ce53527fae3195883650fb7ee277c77ef71631be80c947aac9ff8c6dce24